In [4]:
import requests
import pandas as pd
import scrapy as sc

from pathlib import Path
import os
import zipfile

import time
import re

from clickhouse_driver import Client

from loguru import logger

#Перманентно поменял настройки вывода графиков
from matplotlib import pylab
from pylab import *
pylab.rcParams['figure.figsize'] = (18.0, 6.0)
plt.rcParams.update({'font.size': 13})

#Скрыл вывод предупреждений.
import warnings
warnings.filterwarnings('ignore') #чтобы вернуть: (action='once')

# Глобально снял ограничение на кол-во отображаемых результатов для каждой ячейки ввода кода.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Включил возможность форматировать стили текста с помощью метода printmd()
from IPython.display import Markdown, display, HTML
def printmd(string):
    display(Markdown(string))
    
# Снял ограничение на вывод кол-ва столбцов и ширины колонки.
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', -1)

# Установил формат вывода в таблице на 2 знака после запятой.
pd.options.display.float_format = '{:,.3f}'.format
    
# Добавил функцию вывода таблиц в одну строку, для экономии пространства и улучшения восприятия информации.
def display_side_by_side(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [2]:
#Настроил логер
logger.add('logs/logs.log', format="{time} {level} {message}", filter="my_module", level="INFO")

1

### Extract

In [7]:
#Сделал запрос на скачивание xml-файла из s3 бакета со списком всех zip архивов.
test_req = requests.get('https://s3.amazonaws.com/tripdata')

#Записал файл в pandas датафрейм.
tt = pd.read_xml(test_req.text)

In [9]:
#Отсеял все поля, которые не содержат значения Key(название zip-файла)
table_source = tt[tt['Key'] == tt['Key']].iloc[:, 3:-1].reset_index(drop=True)[:-1]
table_source.head(3)

,Key,LastModified,ETag,Size
0,201306-citibike-tripdata.zip,2018-04-30T13:18:55.000Z,"""b520a12de58eea58a3586f89bfcfbd9d-2""","16,785,103.000"
1,201307-201402-citibike-tripdata.zip,2017-01-18T22:23:25.000Z,"""7b3b260b2ab2e5349320121d04bd821c-22""","178,262,576.000"
2,201307-citibike-tripdata.zip,2017-01-18T22:23:27.000Z,"""dd3e6fd5f91715b31eae72868086c08c-4""","27,074,629.000"


In [5]:
#Создал колонку с url
table_source['url'] =  table_source['Key'].apply(lambda x: 'https://s3.amazonaws.com/tripdata/' + str(x))
table_source.head(3)

,Key,LastModified,ETag,Size,url
0,201306-citibike-tripdata.zip,2018-04-30T13:18:55.000Z,"""b520a12de58eea58a3586f89bfcfbd9d-2""","16,785,103.000",https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip
1,201307-201402-citibike-tripdata.zip,2017-01-18T22:23:25.000Z,"""7b3b260b2ab2e5349320121d04bd821c-22""","178,262,576.000",https://s3.amazonaws.com/tripdata/201307-201402-citibike-tripdata.zip
2,201307-citibike-tripdata.zip,2017-01-18T22:23:27.000Z,"""dd3e6fd5f91715b31eae72868086c08c-4""","27,074,629.000",https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip


In [11]:
logger.info('Сравнение таблицы исходника с актуальной версией')
#Файл etl_log.csv хранит актуальную историю последней выгрузки. 

#Если файл уже существует, то скрипт открывает его и удаляет колонку update
if os.path.exists('etl_log.csv'):
    logger.info('Таблица загрузчика существует.')
    
    table_actual = pd.read_csv('etl_log.csv',sep=";")
    del table_actual['update'] #колонка update указывает следует ли загружать файл
    
    #Объединяются 2 таблицы для построчного сравнения. В результате создается колонка обноваленная колонка update
    table_final=table_source.merge(table_actual,indicator=True,how='left')
    table_final._merge=table_final._merge.eq('both')
    table_final = table_final.rename(columns={'_merge':'update'})
    table_final['update'] = ~table_final['update']

    #Подсчитывается количество изменений и выводиться соответствующее сообщение.
    changes = table_final[table_final['update'] != False]
    
    if len(changes) == 0:
        logger.info('Изменений нет.')
    else:
        logger.info(len(changes) + 'изменений найдено.')
#Если файл не существует, то создает его с указанными по умолчанию значениями.
else:
    table_source['downloaded'] = False #метка был ли скачан файл.
    table_source['extracted'] = False #метка были ли разархивирован файл.
    table_source['ch_uploaded'] = False #метка был ли файл загружен в БД clickhouse.
    table_source['csv'] = 'No' #название csv файла при его существовании.
    
    table_source.to_csv('etl_log.csv',sep=";",index=False)
    
    logger.info('Файл таблицы создан.')

2022-09-14 23:52:28.771 | INFO     | __main__:<cell line: 1>:1 - Сравнение таблицы исходника с актуальной версией
2022-09-14 23:52:28.775 | INFO     | __main__:<cell line: 5>:6 - Таблица загрузчика существует.
2022-09-14 23:52:28.804 | INFO     | __main__:<cell line: 5>:18 - Изменений нет.


In [15]:
table_final.head(3)

,Key,LastModified,ETag,Size,url,downloaded,extracted,ch_uploaded,csv,update
0,201306-citibike-tripdata.zip,2018-04-30T13:18:55.000Z,"""b520a12de58eea58a3586f89bfcfbd9d-2""","16,785,103.000",https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip,True,True,True,201306-citibike-tripdata.csv,False
1,201307-201402-citibike-tripdata.zip,2017-01-18T22:23:25.000Z,"""7b3b260b2ab2e5349320121d04bd821c-22""","178,262,576.000",https://s3.amazonaws.com/tripdata/201307-201402-citibike-tripdata.zip,True,True,True,2013-07 - Citi Bike trip data.csv,False
2,201307-citibike-tripdata.zip,2017-01-18T22:23:27.000Z,"""dd3e6fd5f91715b31eae72868086c08c-4""","27,074,629.000",https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip,True,True,True,2013-07 - Citi Bike trip data.csv,False


In [22]:
# Методы сохранения файлов
def get_file_file(url): #делаем запрос на сам файл
    try:
        r = requests.get(url, stream=True)
        return r
    except:
        print('Нет коннекта при соединении с файлом по указанной ссылке.')
        logger.info('Нет коннекта при соединении с файлом по указанной ссылке.')
        pass


def get_file_name(url): #получаем имя файла
    try:
        name = url.split('/')[-1]  #потрошим ссылку через / и берём оттуда последние данные
        return name
    except:
        return url

def save_file(name, file_object): #сохраняет файл в корневой папке
    try:
        path = './files/'+name
        with open(path, 'bw') as f:
            for chunk in file_object.iter_content(None):
                f.write(chunk)
            print(str(name)+' загружен')
            logger.info(str(name)+' загружен')
    except:
        pass
    time.sleep(5)

In [23]:
# Директория сохранения файлов
dir_path = r'./files/'

# Список файлов в директории
res = os.listdir(dir_path)

In [25]:
#Функция извлекает csv-файл из архива в путь ./files/csv/, принимает атрибуты: имя файла, индекс.
def zip_extractor(target_file, index):
    endswith = '.csv'
    try:
        path = Path(target_file)
        target_file_size = path.stat().st_size
        
        with zipfile.ZipFile(target_file) as z:
            for file in z.namelist():
                if file.endswith(endswith):
                    if (target_file_size) > 0:
                        logger.info("Извлекаемый из архива файл существует: "+file)
                    else:
                        z.extract(file,'./files/csv/')
                logger.info(file+" файл разархивирован")
                if file[1] != "_":
                    table_final.at[index, 'csv'] = file
        table_final.at[index, 'extracted'] = True
    except ValueError as ve:
        logger.info(ve)

In [ ]:
#Итератор проходит по созданной ранее таблице с актуальной версией инструкций экстракции. 
for index, row in table_final.iterrows():
    file = row['url']
    filename = file.split('/')[-1]
    try:
        path = Path(dir_path+filename)
        target_file_size = path.stat().st_size
    except:
        logger.info("Файл скачивается: "+file)
        save_file(get_file_name(file), get_file_file(file))
    else:
        if (target_file_size) > 0:
            logger.info("Файл существует: "+filename)
        else:
            logger.info("Файл скачан не полностью, продолжается скачивание: "+file)
            save_file(get_file_name(file), get_file_file(file))
            
    table_final.at[index, 'downloaded'] = True
    zip_extractor('.\\'+str(path),index)

In [34]:
#Сохранение изменений инструкций экстракции.
table_final.to_csv('etl_log.csv',sep=";",index=False)

### Transform Load

In [37]:
#Функция просеивает данные из одного csv файла, группирует их по условию и загружает в БД в три разные таблицы. Принимает атрибуты: имя файла, индекс.
def load_trip(filename,index):
    try:
        #Читаем csv файл в Dataframe
        df = pd.read_csv('./files/csv/'+filename)
#         df.info()
        
        #Приводим колонки к одному виду
        df.columns = df.columns.str.replace(' ', '')
        df.columns = df.columns.str.lower()
        try:
            df['stop_date'] = pd.to_datetime(df['stoptime'])
        except:
            df = df.rename(columns={'ended_at':'stop_date','ride_id':'bikeid','':'tripduration','member_casual':'gender'})
            df['stop_date'] = pd.to_datetime(df['stop_date'])
            df['started_at'] = pd.to_datetime(df['started_at'])
            df['tripduration'] = (df['stop_date'] - df['started_at']) / np.timedelta64(1, 's')
            df = df.replace('member', 1).replace('casual', 2)
            pass
            
        #Преобразуем тип данных для работы со временем.
        df['stop_date'] = df['stop_date'].astype('datetime64[D]')
    #     df.head(5)
 
        #Группируем датафреймы по дням, следуя условиям задания.
        trips_count_daily = df.groupby('stop_date')['bikeid'].count().to_frame().reset_index().rename(columns={'bikeid':'trips_count','stop_date':'date'})
    #     trips_count_daily.head(10)
        avg_duration_daily = df.groupby('stop_date')['tripduration'].mean().to_frame().reset_index().rename(columns={'tripduration':'avg_trip_dur','stop_date':'date'})
    #     avg_duration_daily.head(10)
        gender_df = df.groupby('stop_date')['bikeid'].count().to_frame()
        

        for sex in df['gender'].unique():
            gender_df['gender_'+str(sex)] = df[df['gender'] == sex].groupby('stop_date')['gender'].count().to_frame()
        gender_daily = gender_df.reset_index().rename(columns={'stop_date':'date'}) 
        del gender_daily['bikeid']
        for sex in df['gender'].unique():
            gender_daily['gender_'+str(sex)] = gender_daily['gender_'+str(sex)].fillna(0).astype(int)  
        if 'gender_0' not in gender_daily.columns:
            gender_daily['gender_0'] = 0
    #     gender_daily.head(5)

        #Создаем коннект к Clickhouse. В clickhouse. Файлы DDL в папке проекта. Для таблиц использовались движки ReplacingMergeTree и AggregatingMergeTree
        client = Client(host='localhost', settings={'use_numpy': True}).from_url('clickhouse://default:@localhost:19000/default') #логин пароль порт по умолчанию, дефолтная БД тоже по умолчанию
    #     result = client.execute("SHOW TABLES")
    #     print(result)

        client.execute('INSERT INTO `default`.gender_daily VALUES', gender_daily.to_dict('records'),types_check=True)
        client.execute('OPTIMIZE TABLE `default`.gender_daily')
    #     client.query_dataframe('SELECT * FROM gender_daily')

        client.execute('INSERT INTO `default`.trips_count_daily VALUES', trips_count_daily.to_dict('records'),types_check=True)
        client.execute('OPTIMIZE TABLE `default`.trips_count_daily')
    #     client.query_dataframe('SELECT * FROM trips_count_daily')

        client.execute('INSERT INTO `default`.avg_duration_daily VALUES', avg_duration_daily.to_dict('records'),types_check=True)
        client.execute('OPTIMIZE TABLE `default`.avg_duration_daily')
    # client.query_dataframe('SELECT * FROM avg_duration_daily')
        table_final.at[index, 'ch_uploaded'] = True
        logger.info('Файл успешно загружен в БД: ' + filename)
    except ValueError as ve:
        logger.info(ve)

In [ ]:
#Итератор выполняют функцию load_trip, тем самым обрабатывает и загружает подготовленные данные в БД.
for index, row in table_final.iterrows():
    uploaded = row['ch_uploaded']
    if uploaded != True:
        load_trip(row['csv'],index)
    else:
        logger.info('Датасет уже существует в БД: ' + row['csv'])

In [39]:
#Сохранение изменений инструкций экстракции.
table_final.to_csv('etl_log.csv',sep=";",index=False)